<a href="https://colab.research.google.com/github/BrianIto/ML_DEN_RNA/blob/main/One_Hot_Encoding_na_proteina.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

ns1 = pd.read_csv('E_protein.csv')

In [ ]:
#Transforma String do vetor do CSV em vetor.
sequences = ns1["sequences"].str.replace("[", "")
sequences = sequences.str.replace("]", "")
sequences = sequences.str.replace("'", "")
sequences = np.array(sequences.str.split(", "))
ns1.sequences = sequences

In [ ]:
#Severa == 1 e Classica == 0
ns1["Label"] = (ns1["outcome"] == "Severa").astype(int)

train_labels = np.array(ns1["Label"])
train_labels

In [ ]:
train_data = np.array(ns1["sequences"])
data_dict = {}

#Cria Dicionário de Frequência
for i, sequence in enumerate(train_data):
  for j, codon in enumerate(sequence):
    if (codon in data_dict):
      data_dict[codon] += 1
    else:
      data_dict[codon] = 1

#Cria Dicionário de Ranking de Aparição
def getMaior(dict):
  max, maxKey = -1, ''
  for key in dict:
    if (max < dict[key]):
      max = dict[key]
      maxKey = key
  return max, maxKey
  
size = len(data_dict.keys())

data_dict_copy = data_dict.copy()
dict_final = {}

for i in range(0, size):
  max, maxKey = getMaior(data_dict_copy)
  del data_dict_copy[maxKey]
  dict_final[maxKey] = i + 1

dict_final

In [ ]:
for i in range(0, ns1["sequences"].shape[0]):
  for j in range(0, len(ns1["sequences"].loc[i])):
    ns1["sequences"].loc[i][j] = dict_final[ns1["sequences"].loc[i][j]]
  

In [ ]:
ns1

In [ ]:
arr = np.array(ns1.sequences)

matriz = []
for i, lista in enumerate(arr):
  matriz_dispersao = np.zeros(71)
  for j, elem in enumerate(lista):
    matriz_dispersao[elem] += 1
  matriz.append(matriz_dispersao.astype(int))

train_data = np.array(matriz)
ns1["matriz_disp"] = matriz
ns1

NameError: ignored

In [ ]:
import tensorflow as tf


train_data = ns1["sequences"]

train_data.shape
#tf.constant(train_data)
train_data

series = ns1["sequences"].copy()
slist = series.to_list()
train_data = np.array(slist)

In [ ]:
arr = np.array(ns1["matriz_disp"].to_list())

totalCodons = arr.shape[1]
totalSequences = arr.shape[0]

x = np.zeros(totalSequences);
y = np.zeros(totalSequences);

for index, sequence in enumerate(arr):
  if (index + 1 < totalSequences):
    differenceMatrix = np.zeros(totalSequences)
    counter = 0
    for i, codon in enumerate(sequence):
      if (i + 1  < totalCodons):
        if (codon != arr[index + 1][i + 1]):
          differenceMatrix[i] += 1
          counter += 1
    print("Posição: " + str(index) + ", Taxa de Igualdade: " + str(100 - (counter / totalSequences)))
    x[index] = index;
    y[index] = 100 - (counter / totalSequences)
  
plt.scatter(x, y)

In [ ]:
from keras import models, Model
from keras import layers 

def build_model():
    inputs = layers.Input(shape=(None, 71)) # placeholder
    x = layers.Dense(128, activation='relu')(inputs)
    out = layers.Dense(1, activation='softmax')( x )

    model = Model(inputs=[inputs], outputs=[out], name="mnits_model")

    model.summary()

    model.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])
    return model
  
modelito = build_model()

history = modelito.fit(train_data[:100], train_labels[:100], epochs=20, batch_size=512, 
                       validation_data=(train_data[100:157], train_labels[100:157]))